In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [22]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [4]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
ed = Website("https://www.futureviewsystems.com/")
ed.links

['/',
 '/finance-transformation-services-and-solutions',
 '/finance-transformation-services-and-solutions',
 '/finance-transformation-services-and-solutions',
 '/platform-financial-planning-and-analysis',
 '/platform-financial-planning-and-analysis',
 '/platform-financial-planning-and-analysis',
 '/case-studies',
 '/integrations',
 '/integrations/netsuite-data-integration-reporting-tool',
 '/integrations/quickbooks-data-forecasting-reporting-tool-integration',
 '/finance-excel-add-in',
 '/financial-reporting-software-tools',
 '/erp-data-extraction-tool',
 '/budgeting-and-forecasting-software',
 '/cpm-software',
 '/board-pack-software',
 '/solutions/finance-teams',
 '/solutions/interim-and-fractional-cfo-service-solutions',
 '/futureview-systems-comparison',
 '/solutions/controllers-and-accounting-teams',
 '/solutions/data-analysts-and-it-solutions',
 '/blog',
 '/resources/white-papers',
 '/resources/webinars',
 '/faq',
 '/case-studies',
 '/about',
 '/announcements',
 '/resources/future

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.futureviewsystems.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/finance-transformation-services-and-solutions
/finance-transformation-services-and-solutions
/finance-transformation-services-and-solutions
/platform-financial-planning-and-analysis
/platform-financial-planning-and-analysis
/platform-financial-planning-and-analysis
/case-studies
/integrations
/integrations/netsuite-data-integration-reporting-tool
/integrations/quickbooks-data-forecasting-reporting-tool-integration
/finance-excel-add-in
/financial-reporting-software-tools
/erp-data-extraction-tool
/budgeting-and-forecasting-software
/cpm-software
/board-pack-software
/solutions/finance-teams
/solutions/interim-and-fractional-cfo-service-solutions
/futureview-systems-comparison
/sol

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://www.futureviewsystems.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.futureviewsystems.com/about'}, {'type': 'careers page', 'url': 'https://www.futureviewsystems.com/careers'}, {'type': 'services page', 'url': 'https://www.futureviewsystems.com/services'}, {'type': 'case studies page', 'url': 'https://www.futureviewsystems.com/case-studies'}, {'type': 'blog page', 'url': 'https://www.futureviewsystems.com/blog'}, {'type': 'demo page', 'url': 'https://www.futureviewsystems.com/demo'}, {'type': 'white papers page', 'url': 'https://www.futureviewsystems.com/resources/white-papers'}, {'type': 'announcements page', 'url': 'https://www.futureviewsystems.com/announcements'}]}
Landing page:
Webpage Title:
Tech-Enabled Finance Solutions and Services - FutureView Systems
Webpage Contents:
Solutions
Finance Transformations
Tailored finance solutions and services
to transform your finance function
Finance Tools and Solutions
FP&A Software implementation for more
detailed reporting, budgeting and fo

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("futureview","https://www.futureviewsystems.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.futureviewsystems.com/about'}, {'type': 'careers page', 'url': 'https://www.futureviewsystems.com/careers'}, {'type': 'services page', 'url': 'https://www.futureviewsystems.com/services'}, {'type': 'case studies page', 'url': 'https://www.futureviewsystems.com/case-studies'}, {'type': 'blog page', 'url': 'https://www.futureviewsystems.com/blog'}, {'type': 'demo page', 'url': 'https://www.futureviewsystems.com/demo'}, {'type': 'resources page', 'url': 'https://www.futureviewsystems.com/resources'}, {'type': 'contact page', 'url': 'https://www.futureviewsystems.com/contact-us'}]}


'You are looking at a company called: futureview\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nTech-Enabled Finance Solutions and Services - FutureView Systems\nWebpage Contents:\nSolutions\nFinance Transformations\nTailored finance solutions and services\nto transform your finance function\nFinance Tools and Solutions\nFP&A Software implementation for more\ndetailed reporting, budgeting and forecasting\nUse Cases\nNo-code Integrations\nNetsuite Reporting\nQuickbooks Reporting\nFinance Excel Add-in\nFinancial Reporting\nERP Data Extraction &\xa0Staging\nBudgeting &\xa0Forecasting\nCorporate Performance (CPM)\nBoard Pack Reporting\nServices\nFractional FP&A\nInterim &\xa0Fractional CFOs\nCompare FutureView\nControllers &\xa0Accounting\nBusiness Intelligence\nResources\nBlog\nWhite Papers\nWebinars\nFAQ\nCase Studies\nAbout Us\nAnnouncements\nFutureView Quarte

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("futureview", "https://www.futureviewsystems.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.futureviewsystems.com/about'}, {'type': 'services page', 'url': 'https://www.futureviewsystems.com/services'}, {'type': 'careers page', 'url': 'https://www.futureviewsystems.com/careers'}, {'type': 'case studies page', 'url': 'https://www.futureviewsystems.com/case-studies'}, {'type': 'blog page', 'url': 'https://www.futureviewsystems.com/blog'}, {'type': 'demo page', 'url': 'https://www.futureviewsystems.com/demo'}, {'type': 'white papers page', 'url': 'https://www.futureviewsystems.com/resources/white-papers'}, {'type': 'resources page', 'url': 'https://www.futureviewsystems.com/resources'}, {'type': 'contact page', 'url': 'https://www.futureviewsystems.com/contact-us'}, {'type': 'announcements page', 'url': 'https://www.futureviewsystems.com/announcements'}]}


```markdown
# FutureView Systems Brochure

## About Us
FutureView Systems is at the forefront of tech-enabled finance solutions and services, dedicated to transforming the finance function of businesses. We empower finance teams with tailored, innovative tools and services backed by decades of executive experience. Our mission is to streamline financial processes while providing actionable insights that drive strategic initiatives.

## Solutions We Offer
- **Finance Transformations**: Customized solutions aimed at revolutionizing the finance function.
- **FP&A Software Implementation**: Streamlined reporting, budgeting, and forecasting.
- **Integrated Finance Tools**: Tools that include:
  - No-code Integrations
  - Netsuite & Quickbooks Reporting
  - Excel Add-ins
  - ERP Data Extraction & Staging
  - Corporate Performance Management (CPM)
  - Board Pack Reporting

## Services
Our comprehensive range of services includes:
- **Fractional FP&A and CFO Services**: Expert financial guidance tailored to your needs.
- **Controllers & Accounting Services**: Ensuring accuracy and transparency in financial reporting.
- **Business Intelligence**: Unlocking insights from data.

## Client Testimonials
"Our budgeting and forecasting process is more efficient and quicker with FutureView."  
— **Brad Schomber**, CFO, Spoonflower

"FutureView has been a key partner to our FP&A team as our business has doubled."  
— **Maddy Devine**, Director, Lucid

"FutureView is an extension of our team that has been super valuable, combining technical expertise with finance professionalism."  
— **Chris Laidley**, CFO, Mursion

## Company Culture
At FutureView, we foster a culture of collaboration, innovation, and continuous improvement. Our team consists of seasoned finance professionals who bring a wealth of technical and industry knowledge. We emphasize the importance of a supportive environment that encourages growth and learning, allowing our employees to thrive on both personal and professional levels.

## Career Opportunities
Join us in shaping the future of finance! We are always on the lookout for passionate individuals who aspire to make a difference in the finance tech industry. Opportunities span across various roles, including finance professionals, technical experts, and support staff. Be a part of a team that values your insights and fosters your growth.

## Contact Us
Ready to transform your finance function?  
[Learn More](#) | [Contact Us](mailto:info@futureview.com)

---

**FutureView Systems**  
**Empowering Finance Teams for Tomorrow**
```


In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("futureview", "https://www.futureviewsystems.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.futureviewsystems.com/about'}, {'type': 'services page', 'url': 'https://www.futureviewsystems.com/services'}, {'type': 'careers page', 'url': 'https://www.futureviewsystems.com/careers'}, {'type': 'case studies page', 'url': 'https://www.futureviewsystems.com/case-studies'}, {'type': 'demo page', 'url': 'https://www.futureviewsystems.com/demo'}, {'type': 'blog page', 'url': 'https://www.futureviewsystems.com/blog'}, {'type': 'resources page', 'url': 'https://www.futureviewsystems.com/resources'}, {'type': 'contact page', 'url': 'https://www.futureviewsystems.com/contact-us'}]}


# FutureView Systems Brochure

## About Us
**FutureView Systems** is a leading provider of tech-enabled finance solutions and services dedicated to transforming finance functions across diverse industries. Our mission is to empower finance teams with tailored solutions that drive business success, and we are backed by decades of finance executive experience. We believe in blending technology with seasoned financial expertise to create an efficient, scalable finance function for our clients.

## Our Solutions
- **Finance Transformations:** Customized finance solutions to enhance your organization's financial capabilities.
- **Finance Tools:** Comprehensive FP&A (Financial Planning and Analysis) software implementations for improved reporting, budgeting, and forecasting.
- **Integrations:** No-code solutions including Netsuite Reporting, Quickbooks Reporting, ERP Data Extraction, and Staging.
- **Corporate Performance Management (CPM):** Streamline Corporate Performance Reporting and Budgeting processes.

### Services Offered
- **Fractional FP&A Services**
- **Interim & Fractional CFO Services**
- **Controllers & Accounting**
- **Business Intelligence**

## Why Choose FutureView?
Our dedicated, "white-glove" services facilitate rapid transformations and deliver insightful financial analytics. Companies like Mursion, Spoonflower, and Lucid have seen significant improvements in their budgeting and forecasting processes through our tailored services and skilled team.

> **Testimonial:**
> "FutureView is an extension of our team in a way that’s been super valuable. They’re not just technical experts, but finance professionals with rich careers." - Chris Laidley, CFO, Mursion

## Company Culture
At FutureView, we value collaboration, innovation, and integrity. Our team is composed of experienced finance professionals who share a common goal: to empower our clients by delivering actionable insights and dynamic solutions. We foster a culture of continuous learning and support, enabling our employees to grow both personally and professionally.

## Careers at FutureView
Join our mission to empower finance teams! We're always looking for passionate individuals eager to make an impact in the finance industry. Positions range from finance professionals to tech specialists, all working towards transforming our clients' financial functions. 

### Current Opportunities:
- FP&A Analysts
- Financial Consultants
- Technical Integrations Specialists
- Client Success Managers

## Contact Us
Ready to transform your finance function? Reach out to us today!

- **Website:** [FutureView Systems](https://www.futureviewsystems.com)
- **Email:** info@futureviewsystems.com
- **Phone:** (555) 123-4567

### Upcoming Events
Join our upcoming webinar: **"Making Your Budget a Sprint, Not a Marathon."** 

---

**FutureView Systems**: Where Finance Software Meets Stellar Services! Transform your finance function today!